In [ ]:
# Hyperparameter grids
random_state_grid = [251]
n_estimators_grid = [100]

# Store MSE and hyperparameter combinations
rmse_results_rf = []

# Run RandomForestRegressor with varying max_features
for random_state, n_estimators in itertools.product(
        random_state_grid, n_estimators_grid):

    rf_model = RandomForestRegressor(n_estimators=n_estimators,
                                         random_state=random_state, n_jobs=8, max_features=n_features//3,max_depth=3)

    predictions_rf = []
    actual_values_rf = []

    tscv = TimeSeriesSplit(n_splits=30,  gap=0)

    for train_index, test_index in tscv.split(df):
        train, test = df.iloc[train_index], df.iloc[test_index]
        X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
        X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

        rf_model.fit(X_train, y_train)
        y_pred_rf = rf_model.predict(X_test)

        predictions_rf.extend(y_pred_rf)
        actual_values_rf.extend(y_test)

    rmse_rf = np.sqrt(mean_squared_error(actual_values_rf, predictions_rf))
    rmse_results_rf.append([random_state, n_estimators,  rmse_rf])

    # Print results
    print(f'RF - Random state: {random_state}, Number of estimators: {n_estimators},  RMSE: {rmse_rf}')

from sklearn.tree import plot_tree
plt.figure(figsize=(20, 10))
plot_tree(rf_model.estimators_[0], max_depth=3, filled=True, fontsize=10, feature_names=df.drop(columns=['CPIAUCSL']).columns)
plt.show()

